# Import records from Google Sheets

The curation of metadata records is done in a [Google Sheets document](https://docs.google.com/spreadsheets/d/1acNnSzn8XrCDFrxf4joqSAUsAQoOgHEvBU-FIt9p9kU/edit?usp=sharing) with five sheets (tabs), one per document type.

This notebook will read the current contents of the sheet into a Pandas dataframe. The records are converted into Python dictionaries and then saved into JSONL files, split into separate train and test subsets.

In [3]:
# Read the metadata from the Google Sheets document into Pandas dataframes

import pandas as pd
import urllib.parse

DOC_ID = "1acNnSzn8XrCDFrxf4joqSAUsAQoOgHEvBU-FIt9p9kU"  # Google Sheets id
LANGUAGES = ('fi', 'sv', 'en', 'se')
#go through only articles for now
SHEET_NAMES = {
 #   "thes": "Theses",
  #  "docthes": "Doctoral theses",
   # "report": "Reports",
    #"book": "Books",
    "article": "Articles"
}

def read_sheet(doc_id, sheet_name, sheet_id):
    csv_url = f"https://docs.google.com/spreadsheets/d/{doc_id}/gviz/tq?tqx=out:csv&sheet={urllib.parse.quote(sheet_name)}"
    df = pd.read_csv(csv_url, dtype=str, na_filter=False)
    # add a new "rowid" column, with values like "thes37", in between other housekeeping columns
    df["rowid"] = df.index + 2
    df["rowid"] = df["rowid"].apply(lambda x: f"{sheet_id}{x}")
    df.insert(3, "rowid", df.pop("rowid"))
    # restrict to monolingual records in the languages we are interested in
    df = df.loc[df["language"].isin(LANGUAGES)]
    df.head(2)
    return df

sheets = {sheet_id: read_sheet(DOC_ID, sheet_name, sheet_id)
          for sheet_id, sheet_name in SHEET_NAMES.items()}
for i, (sheet_id, sheet_data) in enumerate(sheets.items()): 
    if i < 3: 
        print(sheet_id, sheet_data) 
    else: 
        break

article     Repository                         Collection  \
0        Doria         ÅA / Kirjoja, sarjoja, ym.   
1        Doria         ÅA / Kirjoja, sarjoja, ym.   
2        Doria         ÅA / Kirjoja, sarjoja, ym.   
3         Taju                         Artikkelit   
4      Theseus                 Laurea / Julkaisut   
..         ...                                ...   
184   OuluRepo  Oulun yliopisto / Avoin saatavuus   
185   OuluRepo  Oulun yliopisto / Avoin saatavuus   
186   OuluRepo  Oulun yliopisto / Avoin saatavuus   
187   OuluRepo  Oulun yliopisto / Avoin saatavuus   
188   OuluRepo  Oulun yliopisto / Avoin saatavuus   

                                                   url       rowid  \
0    https://www.doria.fi/bitstream/handle/10024/18...    article2   
1    https://www.doria.fi/bitstream/handle/10024/18...    article3   
2    https://www.doria.fi/bitstream/handle/10024/17...    article4   
3    https://taju.uniarts.fi/bitstream/handle/10024...    article5   
4    

In [4]:
# Calculate language statistics for each sheet (document type) and combine into an overview

language_counts = {sheet_id: sheet["language"].value_counts().rename(sheet_id).astype(int)
                   for sheet_id, sheet in sheets.items()}

langstat = pd.concat(language_counts.values(), axis=1).fillna(0).astype(int)
langstat['TOTAL'] = langstat.sum(axis=1)
langstat.loc['TOTAL'] = langstat.sum()
langstat

,article,TOTAL
language,,
en,79,79
fi,70,70
sv,33,33
se,7,7
TOTAL,189,189


In [5]:
# Calculate by-repository statistics for each sheet (document type) and combine into an overview

repo_counts = {sheet_id: sheet["Repository"].value_counts().rename(sheet_id).astype(int)
               for sheet_id, sheet in sheets.items()}

repostat = pd.concat(repo_counts.values(), axis=1).fillna(0).astype(int)
repostat['TOTAL'] = repostat.sum(axis=1)
repostat.loc['TOTAL'] = repostat.sum()
repostat

,article,TOTAL
Repository,,
Theseus,113,113
Taju,20,20
Doria,16,16
Osuva,16,16
Kaisu,13,13
OuluRepo,7,7
Julkari,4,4
TOTAL,189,189


In [6]:
# Calculate by-COAR-type statistics for each sheet (document type) and combine into an overview

coar_counts = {sheet_id: sheet["type_coar"].value_counts().rename(sheet_id).astype(int)
               for sheet_id, sheet in sheets.items()}

coarstat = pd.concat(coar_counts.values(), axis=1).fillna(0).astype(int)
coarstat['TOTAL'] = coarstat.sum(axis=1)
coarstat.loc['TOTAL'] = coarstat.sum()
coarstat

,article,TOTAL
type_coar,,
journal article,85,85
research article,45,45
newspaper article,21,21
conference paper,17,17
blog post,13,13
book review,4,4
editorial,3,3
review article,1,1
TOTAL,189,189


In [7]:
# every 1 out of TRAIN_TEST_SPLIT_FACTOR records (on average) will be placed in test set
TRAIN_TEST_SPLIT_FACTOR = 4

import json
import re
import zlib

def url_to_id(url):
    return re.sub(r"(.+)/bitstream/handle/(\d+)/(\d+)/.*", r"\1/handle/\2/\3", url)

def convert_colname(col):
    if col in ('rowid', 'url'):
        return col
    elif col == 'Repository':
        return 'repository'
    return 'dc.' + '.'.join(col.split('/'))

def filter_vals(vals):
    # remove values marked with [], meaning that they can't be directly inferred from the document
    # this works on lists of values as well as individual (string) values
    if isinstance(vals, list):
        return [v for v in vals if v and not (v.startswith('[') and v.endswith(']'))]

    val = vals
    if not (val.startswith('[') and val.endswith(']')):
        return val
    else:
        return None

def row_to_dict(row):
    gt = {}

    # language
    gt["language"] = row["language"].strip()

    # title
    if (title := filter_vals(row["title"].strip())):
        gt["title"] = title

    # alt_title
    if (alt_titles := filter_vals(row["alt_title"].strip().split("\n"))):
        gt["alt_title"] = alt_titles

    # creator
    if (creators := filter_vals(row["creator"].strip().split("\n"))):
        gt["creator"] = creators

    # year
    if (issued := filter_vals(row["year"].strip())):
        gt["year"] = issued[:4]  # only include year part

    # publisher
    if (publishers := filter_vals(row["publisher"].strip().split("\n"))):
        gt["publisher"] = publishers

    # DOI
    if (doi := filter_vals(row["doi"].strip())):
        gt["doi"] = doi
        
    # e-ISBN
    if (eisbns := filter_vals(row["e-isbn"].strip().split("\n"))):
        gt["e-isbn"] = [isbn.replace('-', '') for isbn in eisbns]  # strip dashes in ISBNs

    # p-ISBN
    if (pisbns := filter_vals(row["p-isbn"].strip().split("\n"))):
        gt["p-isbn"] = [isbn.replace('-', '') for isbn in pisbns]  # strip dashes in ISBNs

    # e-ISSN
    if (eissn := filter_vals(row["e-issn"].strip())):
        gt["e-issn"] = eissn

    # p-ISSN
    if (pissn := filter_vals(row["p-issn"].strip())):
        gt["p-issn"] = pissn

    # COAR resource type
    if (coartype := filter_vals(row["type_coar"].strip())):
        gt["type_coar"] = coartype
    
    return {"repository": row["Repository"].strip(),
            "url": row["url"].strip(),
            "id": url_to_id(row["url"].strip()),
            "rowid": row["rowid"].strip(),
            "ground_truth": gt}

def is_test_record(rec):
    """deterministically select, based on the record ID, whether a record goes into the train or test set"""
    return 3 * zlib.crc32(rec['id'].encode('utf-8')) % TRAIN_TEST_SPLIT_FACTOR == 1

for sheet_id, sheet in sheets.items():
    for lang in LANGUAGES:
        df = sheet[sheet['language'] == lang]
        records = [row_to_dict(row) for _, row in df.iterrows()]
        with (open(f"../metadata/{sheet_id}-{lang}-train.jsonl", "w") as trainfile,
              open(f"../metadata/{sheet_id}-{lang}-test.jsonl", "w") as testfile):
            ntrain = ntest = 0
            for rec in records:
                if is_test_record(rec):
                    outfile = testfile
                    subset = 'test'
                    ntest += 1
                else:
                    outfile = trainfile
                    subset = 'train'
                    ntrain += 1
                header = {'doctype': sheet_id, 'subset': subset}
                json.dump(header | rec, outfile)
                outfile.write("\n")
        if records:
            print(f"{sheet_id}-{lang}:\twrote {len(records)} records ({ntrain} train, {ntest} test ({100*ntest/(ntrain+ntest):.1f} %))")

article-fi:	wrote 70 records (57 train, 13 test (18.6 %))
article-sv:	wrote 33 records (28 train, 5 test (15.2 %))
article-en:	wrote 79 records (64 train, 15 test (19.0 %))
article-se:	wrote 7 records (6 train, 1 test (14.3 %))


In [8]:
# Generate statistics about the use of metadata fields

from collections import defaultdict, Counter
import datetime
import glob
from statistics import mean
from tabulate import tabulate


langstat_table = tabulate(langstat, headers='keys', tablefmt='github')
repostat_table = tabulate(repostat, headers='keys', tablefmt='github')
coarstat_table = tabulate(coarstat, headers='keys', tablefmt='github')

metadata_files = glob.glob("../metadata/*.jsonl")

field_counts = defaultdict(Counter)  # key1: doctype, key2: field
doc_counts = Counter()
field_types = {}  # key: field, val: S (single) or M (multiple)
field_nvals = defaultdict(list)  # key: field, val: list of number of values

for mdfile in sorted(metadata_files):
    with open(mdfile) as infile:
        for line in infile:
            rec = json.loads(line)
            # flatten the record: include the ground_truth fields at the top level
            combined_rec = rec | rec["ground_truth"]
            del combined_rec["ground_truth"]
            
            for fld in combined_rec:
                field_counts[rec['doctype']][fld] += 1
                field_types[fld] = 'M' if isinstance(combined_rec[fld], list) else 'S'
                if field_types[fld] == 'M':
                    field_nvals[fld].append(len(combined_rec[fld]))
            doc_counts[rec['doctype']] += 1

data = []

def format_value(val):
    if not val:
        return '-'
    return "{:.0%}".format(val)

for fld in list(field_types.keys()):

    if field_types[fld] == 'M':
        mean_val = mean(field_nvals[fld] or [0])
        max_val = max(field_nvals[fld] or [0])
        ftype = f"M ({mean_val:.1f}/{max_val})"
    else:
        ftype = 'S'


    row = {
        'Field': fld,
        'Type': ftype
    }
    for doctype in SHEET_NAMES.keys():
        row[doctype] = format_value(field_counts[doctype][fld] / doc_counts[doctype])
    data.append(row)

# Create a Pandas DataFrame from the data
df = pd.DataFrame(data)

# Convert DataFrame to markdown table
field_table = tabulate(df, headers='keys', tablefmt='github', showindex=False)

timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

statfile = f"""# Statistics about metadata

Automatically generated {timestamp}

Type is either:
 * S: single-value
 * M: multi-value, number of values given as (mean/max)

Percentages represent the coverage of a field in a subset. 100% coverage means the field is always present.

## Document counts by language and document type

{langstat_table}

## Document counts by repository and document type

{repostat_table}

## Document counts by COAR resource type and document type

{coarstat_table}

## Metadata coverage by document type

{field_table}
"""

print(statfile)

with open('../statistics.md', 'w') as outf:
    print(statfile, file=outf)

# Statistics about metadata

Automatically generated 2025-01-02 17:32:41

Type is either:
 * S: single-value
 * M: multi-value, number of values given as (mean/max)

Percentages represent the coverage of a field in a subset. 100% coverage means the field is always present.

## Document counts by language and document type

| language   |   article |   TOTAL |
|------------|-----------|---------|
| en         |        79 |      79 |
| fi         |        70 |      70 |
| sv         |        33 |      33 |
| se         |         7 |       7 |
| TOTAL      |       189 |     189 |

## Document counts by repository and document type

| Repository   |   article |   TOTAL |
|--------------|-----------|---------|
| Theseus      |       113 |     113 |
| Taju         |        20 |      20 |
| Doria        |        16 |      16 |
| Osuva        |        16 |      16 |
| Kaisu        |        13 |      13 |
| OuluRepo     |         7 |       7 |
| Julkari      |         4 |       4 |
| TOTAL     